In [1]:
#!pip install ../input/mtcnn-package/mtcnn-0.1.0-py3-none-any.whl

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.patches import Rectangle # For drawing rectangle around faces

import os
import sys

import keras
from keras import Model, Sequential
from keras.layers import *
from keras.optimizers import *

import torch

from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

import cv2
from mtcnn import MTCNN

from tqdm.notebook import tqdm
import random

import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


First we define our directory paths and directory lists - including the directory where we will save our train and test images that we extract from the videos. 

In [2]:
train_sample_videos_path = '../input/train_sample_videos/'
test_videos_path = '../input/test_videos/'
train_sample_images_path = "../input/train_sample_images/" # path to save train images to
test_images_path = "../input/test_images/" # path to save test images to

train_sample_videos = os.listdir(train_sample_videos_path) # List train vids
test_videos = os.listdir(test_videos_path) # List test vids

Now we loop round all the videos in our directory to extract images for each video.

**TO DO:**

 - **Be able to define the number of frames to extract, and where these frames are (e.g. frame 1,5,10,... or randomly select frame numbers)**
 - **Have file path as an arg so we can define train or test file paths**

In [3]:
# Extract images
for i in range(0, 5): # Edited out len(train_sample_videos) for dev
    file_name = train_sample_videos[i] # file name with .ext
    vid_name = file_name.split('.')[0] # file name without .ext
    file_path = train_sample_videos_path + file_name # full file path

    cap = cv2.VideoCapture(file_path)
    total_frames = cap.get(7)
    cap.set(1, 100)
    ret, frame = cap.read()
    cv2.imwrite(os.path.join(train_sample_images_path, vid_name + '.jpg'), frame) # Save frame as image
    
    cv2.destroyAllWindows()
    cap.release()

In [4]:
train_sample_images = os.listdir(train_sample_images_path) # List newly created training images
detector = MTCNN()

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
for image in range(0, len(train_sample_images)):
    image_name = train_sample_images[image].split('.')[0] # Get image name without .ext
    
    # Read image and detect faces
    frame = cv2.imread(train_sample_images_path + train_sample_images[image])
    result = detector.detect_faces(frame)

    for face in range(0, len(result)):    
        startX, startY, endX, endY = result[face]['box'] # Get box coordinates
        crop_img = frame[startY:startY + endY, startX:startX + endX] # extract just the face
        cv2.imwrite(train_sample_images_path + image_name + str(face) + '.jpg', crop_img)
    os.remove(train_sample_images_path + train_sample_images[image])

In [ ]:
for face in range(0,len(result)):    
    startX, startY, endX, endY = result[face]['box'] # Get box coordinates
    crop_img = image[startY:startY + endY, startX:startX + endX] # extract just the face
    cv2.imwrite(train_sample_images_path + 'aapnvogymq' + str(face) + '.jpg', crop_img)

In [ ]:
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# Load face detector
mtcnn = MTCNN(device=device).eval()

# Load facial recognition model
resnet = InceptionResnetV1(pretrained='vggface2', num_classes=2, device=device).eval()

In [ ]:
# Get all test videos
filenames = glob.glob('../input/train_sample_videos/*.mp4')

# Number of frames to sample (evenly spaced) from each video
n_frames = 10

X = []
with torch.no_grad():
    for i, filename in enumerate(filenames):
        print(f'Processing {i+1:5n} of {len(filenames):5n} videos\r', end='')
        
        # Create video reader and find length
        v_cap = cv2.VideoCapture(filename)
        v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))
            
        # Pick 'n_frames' evenly spaced frames to sample
        sample = np.linspace(0, v_len - 1, n_frames).round().astype(int)
        imgs = []
        for j in range(v_len):
            success, vframe = v_cap.read()
            vframe = cv2.cvtColor(vframe, cv2.COLOR_BGR2RGB)
            if j in sample:
                imgs.append(Image.fromarray(vframe))
        v_cap.release()
            
        # Pass image batch to MTCNN as a list of PIL images
        faces = mtcnn(imgs)
            
        # Filter out frames without faces
        faces = [f for f in faces if f is not None]
        faces = torch.stack(faces).to(device)
            
        # Generate facial feature vectors using a pretrained model
        embeddings = resnet(faces)
            
        # Calculate centroid for video and distance of each face's feature vector from centroid
        centroid = embeddings.mean(dim=0)
        X.append((embeddings - centroid).norm(dim=1).cpu().numpy())

In [ ]:
embeddings